In [2]:
import pandas as pd
from read_db.CH import Getch
import pandahouse
from datetime import date,timedelta, datetime
import telegram
import seaborn as sns
import matplotlib.pyplot as plt
import io
sns.set(style ='ticks')

- Динамика ключевых метрик
- Графики и файлы
- в 11:00
- Активные пользователи (лента, мессенджер)
- Лайки, просмотры, CTR
- Отправленные сообщения
- Новые юзеры

In [87]:
data = Getch(
    '''
    select   if(t1.user_id = 0,t2.user_id,t1.user_id) as user_id
            , if(t1.day ='1970.01.01',t2.first_date_time, t1.first_date_time) as first_date_time
            , if(t1.day ='1970.01.01',t2.day,t1.day) as day
            , t1.like as like
            , t1.view as view
            , t1.CTR as CTR
            , t2.send_message as send_message
            , t3.recieve_message as recieve_message
    from 
    (select user_id, toDate(time) as day
            ,min(time) as first_date_time
            ,countIf(post_id,action ='like') as like
            ,countIf(post_id,action='view') as view
            ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
    from simulator_20220520.feed_actions
    group by user_id, day) as t1

    --отправка сообщений
    full join
    (select user_id,toDate(time) as day
            ,count(reciever_id) as send_message
            ,min(time) as first_date_time
    from simulator_20220520.message_actions
    group by user_id,day) as t2
    on t1.user_id = t2.user_id and t1.day = t2.day

    --получение сообщений
    full join
    (select reciever_id,toDate(time) as day
            ,count(user_id) as recieve_message
    from simulator_20220520.message_actions
    group by reciever_id,day) as t3
    on if(t1.user_id = 0,t2.user_id,t1.user_id) = t3.reciever_id 
    and if(t1.day ='1970.01.01',t2.day,t1.day) = t3.day
    
    where day > today()-8
    order by t1.day
    '''
    ).df
data.head(2)

,user_id,first_date_time,day,like,view,CTR,send_message,recieve_message
0,117734,2022-05-26 13:05:00,2022-05-26,0,0,0.0,12,8
1,120776,2022-05-24 22:35:03,2022-05-24,0,0,0.0,3,0


In [88]:
data.shape

(161067, 8)

In [89]:
data

,user_id,first_date_time,day,like,view,CTR,send_message,recieve_message
0,117734,2022-05-26 13:05:00,2022-05-26,0,0,0.000000,12,8
1,120776,2022-05-24 22:35:03,2022-05-24,0,0,0.000000,3,0
2,121618,2022-05-27 08:15:01,2022-05-27,0,0,0.000000,6,16
3,114871,2022-05-21 08:05:02,2022-05-21,0,0,0.000000,7,7
4,3736,2022-05-23 03:20:04,2022-05-23,0,0,0.000000,1,4
...,...,...,...,...,...,...,...,...
161062,164342,2022-05-27 08:30:41,2022-05-27,5,27,0.185185,0,0
161063,38751,2022-05-27 09:35:12,2022-05-27,1,12,0.083333,0,0
161064,10053,2022-05-27 00:00:00,2022-05-27,4,36,0.111111,0,0
161065,57415,2022-05-27 09:55:09,2022-05-27,1,9,0.111111,0,0


In [90]:
data = data.groupby('day', as_index = False).agg({'user_id':'nunique',
                                           'like':'sum',
                                           'view':'sum',
                                           'send_message':'sum',
                                           'recieve_message':'sum'})
data['diff'] = data['send_message'] - data['recieve_message']
data

,day,user_id,like,view,send_message,recieve_message,diff
0,2022-05-20,20517,112224,537657,12900,12837,63
1,2022-05-21,20717,116868,564111,12888,12528,360
2,2022-05-22,20079,102580,498156,12900,12842,58
3,2022-05-23,20462,119012,574585,12899,12812,87
4,2022-05-24,21364,135958,675874,12901,12812,89
5,2022-05-25,21821,113263,583467,12899,12846,53
6,2022-05-26,21642,100663,522682,12901,12806,95
7,2022-05-27,14465,46552,232478,5373,5372,1


In [93]:
data = Getch(
    '''
    with t1 as (select user_id, toDate(time) as day
                    ,min(time) as first_date_time
                    ,countIf(post_id,action ='like') as like
                    ,countIf(post_id,action='view') as view
                    ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
                from simulator_20220520.feed_actions
                group by user_id, day
    ),
    t2 as (select user_id,toDate(time) as day
                ,count(reciever_id) as send_message
                ,min(time) as first_date_time
            from simulator_20220520.message_actions
            group by user_id,day
    ),
    t3 as (select reciever_id,toDate(time) as day
                ,count(user_id) as recieve_message
            from simulator_20220520.message_actions
            group by reciever_id,day
    )
    select   if(t1.user_id = 0,t2.user_id,t1.user_id) as user_id
            ,if(user_id = 0,t3.reciever_id,user_id) as reciever_id
            , if(t1.day ='1970.01.01',t2.first_date_time, t1.first_date_time) as first_date_time
            , if(t1.day ='1970.01.01',t2.day,t1.day) as day
            , t1.like as like
            , t1.view as view
            , t1.CTR as CTR
            , t2.send_message as send_message
            , t3.recieve_message as recieve_message
    from t1
    --отправка сообщений
    full join t2 on t1.user_id = t2.user_id and t1.day = t2.day

    --получение сообщений
    full join t3 on if(t1.user_id = 0, t2.user_id, t1.user_id) = t3.reciever_id 
    and if(t1.day ='1970.01.01',t2.day,t1.day) = t3.day
    
    where day > today()-8
    order by t1.day
    '''
    ).df
data.shape

(161091, 9)

In [104]:
data[(data['day'] == '2022-05-27')].head(1000)

,user_id,reciever_id,first_date_time,day,like,view,CTR,send_message,recieve_message
2,121618,121618,2022-05-27 08:15:01,2022-05-27,0,0,0.000000,6,16
22,15336,15336,2022-05-27 07:55:02,2022-05-27,0,0,0.000000,9,13
52,16753,16753,2022-05-27 02:30:04,2022-05-27,0,0,0.000000,6,8
69,118439,118439,2022-05-27 00:20:00,2022-05-27,0,0,0.000000,5,7
112,115099,115099,2022-05-27 11:15:00,2022-05-27,0,0,0.000000,9,0
...,...,...,...,...,...,...,...,...,...
147597,57305,57305,2022-05-27 08:20:01,2022-05-27,6,32,0.187500,0,0
147598,36957,36957,2022-05-27 07:39:49,2022-05-27,3,15,0.200000,0,0
147599,16237,16237,2022-05-27 02:25:50,2022-05-27,2,13,0.153846,0,0
147600,145157,145157,2022-05-27 02:09:52,2022-05-27,2,18,0.111111,0,0


In [97]:
data[data['view']>0].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,19129
1,2022-05-21,18613
2,2022-05-22,18572
3,2022-05-23,19072
4,2022-05-24,19211
5,2022-05-25,20315
6,2022-05-26,20187
7,2022-05-27,13804


In [99]:
data[data['send_message']>0].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,1569
1,2022-05-21,2373
2,2022-05-22,1681
3,2022-05-23,1612
4,2022-05-24,2476
5,2022-05-25,1739
6,2022-05-26,1662
7,2022-05-27,753


In [102]:
data[(data['send_message']>0)&(data['view']>0)].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,181
1,2022-05-21,269
2,2022-05-22,174
3,2022-05-23,222
4,2022-05-24,323
5,2022-05-25,234
6,2022-05-26,207
7,2022-05-27,68


In [96]:
data_new = data.groupby('day', as_index = False).agg({'user_id':'nunique',
                                            'reciever_id':'nunique',
                                           'like':'sum',
                                           'view':'sum',
                                           'send_message':'sum',
                                           'recieve_message':'sum'})
data_new['diff'] = data_new['send_message'] - data_new['recieve_message']
data_new

,day,user_id,reciever_id,like,view,send_message,recieve_message,diff
0,2022-05-20,20517,20517,112224,537657,12900,12837,63
1,2022-05-21,20717,20717,116868,564111,12888,12528,360
2,2022-05-22,20079,20079,102580,498156,12900,12842,58
3,2022-05-23,20462,20462,119012,574585,12899,12812,87
4,2022-05-24,21364,21364,135958,675874,12901,12812,89
5,2022-05-25,21821,21821,113263,583467,12899,12846,53
6,2022-05-26,21642,21642,100663,522682,12901,12806,95
7,2022-05-27,14489,14489,46814,233875,5375,5374,1


In [141]:
data = Getch(
    '''
    with t1 as (select user_id, toDate(time) as day
                    ,min(time) as first_date_time
                    ,countIf(post_id,action ='like') as like
                    ,countIf(post_id,action='view') as view
                    ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
                from simulator_20220520.feed_actions
                group by user_id, day
    ),
    --отправка сообщений
    t2 as (select user_id,toDate(time) as day
                ,count(reciever_id) as send_message
                ,min(time) as first_date_time
            from simulator_20220520.message_actions
            group by user_id,day
    ),
    --получение сообщений
    t3 as (select reciever_id,toDate(time) as day
                ,count(user_id) as recieve_message
            from simulator_20220520.message_actions
            group by reciever_id,day
    )
    select   user_id
            , if(user_id = 0,t3.reciever_id,user_id) as reciever_id
            , user_id_news
            , user_id_message
            , user_id_news_message
            , t3.reciever_id as user_reciever_id
            , first_date_time
            , t.day
            , like
            , view
            , CTR
            , send_message
            , recieve_message
    from 
    (select if(t1.user_id = 0,t2.user_id,t1.user_id) as user_id
            , if(t1.user_id = 0, 0, 1) as user_id_news
            , if(t2.user_id  = 0, 0, 1) as user_id_message
            , if(t1.user_id > 0 and t2.user_id > 0, 1,0) as user_id_news_message
            , if(t1.day ='1970.01.01',t2.first_date_time, t1.first_date_time) as first_date_time
            , if(t1.day ='1970.01.01',t2.day,t1.day) as day
            , t1.like as like
            , t1.view as view
            , t1.CTR as CTR
            , t2.send_message as send_message
    from t1
    full join t2 on t1.user_id = t2.user_id and t1.day = t2.day) as t
    
    full join t3 on t.user_id = t3.reciever_id  and t.day = t3.day
    
    where day > today()-8
    order by t.day
    '''
    ).df
data.shape

(162182, 13)

In [142]:
data

,user_id,reciever_id,user_id_news,user_id_message,user_id_news_message,user_reciever_id,first_date_time,day,like,view,CTR,send_message,recieve_message
0,14764,14764,1,0,0,0,2022-05-20 12:35:09,2022-05-20,6,28,0.214286,0,0
1,51633,51633,1,0,0,0,2022-05-20 04:25:02,2022-05-20,7,18,0.388889,0,0
2,146065,146065,1,0,0,0,2022-05-20 00:09:09,2022-05-20,6,17,0.352941,0,0
3,11595,11595,1,0,0,0,2022-05-20 09:45:16,2022-05-20,4,21,0.190476,0,0
4,32030,32030,1,0,0,0,2022-05-20 21:34:16,2022-05-20,7,19,0.368421,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162177,60429,60429,1,0,0,0,2022-05-27 00:20:46,2022-05-27,2,13,0.153846,0,0
162178,144892,144892,1,0,0,0,2022-05-27 03:28:56,2022-05-27,4,14,0.285714,0,0
162179,38747,38747,1,0,0,0,2022-05-27 01:24:24,2022-05-27,4,20,0.200000,0,0
162180,156556,156556,1,0,0,0,2022-05-27 02:25:05,2022-05-27,0,15,0.000000,0,0


In [144]:
data_new = data.groupby('day', as_index = False).agg({'user_id':'nunique',
                                                      'user_id_news':'sum',
                                                      'user_id_message':'sum',
                                                      'user_id_news_message':'sum',
                                                      'reciever_id':'nunique',
                                                       'like':'sum',
                                                       'view':'sum',
                                                       'send_message':'sum',
                                                       'recieve_message':'sum'})
data_new['diff'] = data_new['send_message'] - data_new['recieve_message']
#data_new['uniq']
data_new

,day,user_id,user_id_news,user_id_message,user_id_news_message,reciever_id,like,view,send_message,recieve_message,diff
0,2022-05-20,20517,19129.0,1569.0,181.0,20517,112224,537657,12900,12837,63
1,2022-05-21,20717,18613.0,2373.0,269.0,20717,116868,564111,12888,12528,360
2,2022-05-22,20079,18572.0,1681.0,174.0,20079,102580,498156,12900,12842,58
3,2022-05-23,20462,19072.0,1612.0,222.0,20462,119012,574585,12899,12812,87
4,2022-05-24,21364,19211.0,2476.0,323.0,21364,135958,675874,12901,12812,89
5,2022-05-25,21821,20316.0,1739.0,234.0,21821,113263,583467,12899,12846,53
6,2022-05-26,21642,20187.0,1662.0,207.0,21642,100663,522682,12901,12806,95
7,2022-05-27,15580,14738.0,934.0,92.0,15580,50900,254753,6055,6040,15


In [111]:
data[(data['user_id_message']>0)&(data['user_reciever_id']>0)].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,987
1,2022-05-21,1448
2,2022-05-22,1060
3,2022-05-23,1031
4,2022-05-24,1527
5,2022-05-25,1123
6,2022-05-26,1092
7,2022-05-27,566


In [112]:
data[(data['user_id_message']>0)|(data['user_reciever_id']>0)].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,1569
1,2022-05-21,2377
2,2022-05-22,1682
3,2022-05-23,1616
4,2022-05-24,2477
5,2022-05-25,1742
6,2022-05-26,1665
7,2022-05-27,859


In [113]:
data[(data['user_id_message']>0)].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,1569
1,2022-05-21,2373
2,2022-05-22,1681
3,2022-05-23,1612
4,2022-05-24,2476
5,2022-05-25,1739
6,2022-05-26,1662
7,2022-05-27,858


In [114]:
data[(data['user_reciever_id']>0)].groupby('day', as_index = False).agg({'user_id':'count'})

,day,user_id
0,2022-05-20,987
1,2022-05-21,1452
2,2022-05-22,1061
3,2022-05-23,1035
4,2022-05-24,1528
5,2022-05-25,1126
6,2022-05-26,1095
7,2022-05-27,567


In [124]:
data.groupby('day', as_index = False)[['user_id_news','user_id_message']].apply(lambda x: x.count() >0)

,day,user_id_news,user_id_message
0,2022-05-20,True,True
1,2022-05-21,True,True
2,2022-05-22,True,True
3,2022-05-23,True,True
4,2022-05-24,True,True
5,2022-05-25,True,True
6,2022-05-26,True,True
7,2022-05-27,True,True


In [145]:
data_2 = Getch(
    '''
    select 
    toDate(time) as day
    ,count(distinct user_id)
    ,countIf(post_id,action ='like') as like
    ,countIf(post_id,action='view') as view
    ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
     from simulator_20220520.feed_actions
     group by day
   
    '''
    ).df
data_2

,day,uniqExact(user_id),like,view,CTR
0,2022-03-28,1198,1880,8208,0.229045
1,2022-03-29,2325,13305,60906,0.218451
2,2022-03-30,3388,26996,125230,0.215571
3,2022-03-31,4121,32498,150516,0.215911
4,2022-04-01,4917,37450,179308,0.208859
...,...,...,...,...,...
56,2022-05-23,19072,119012,574585,0.207127
57,2022-05-24,19211,135958,675874,0.201159
58,2022-05-25,20316,113263,583467,0.194121
59,2022-05-26,20187,100663,522682,0.192589


In [153]:
data_3 = Getch(
    '''
    select 
    toDate(time) as day
    ,source
    ,count(distinct user_id) as DAU
    ,countIf(post_id,action ='like') as like
    ,countIf(post_id,action='view') as view
    ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
     from simulator_20220520.feed_actions
     group by day, source
     order by day, source
    '''
    ).df
data_3

,day,source,DAU,like,view,CTR
0,2022-03-28,ads,670,1058,4694,0.225394
1,2022-03-28,organic,528,822,3514,0.233921
2,2022-03-29,ads,1161,6642,30270,0.219425
3,2022-03-29,organic,1164,6663,30636,0.217489
4,2022-03-30,ads,1667,13216,60814,0.217318
...,...,...,...,...,...,...
117,2022-05-25,organic,11955,66187,341903,0.193584
118,2022-05-26,ads,8275,41104,213075,0.192909
119,2022-05-26,organic,11912,59559,309607,0.192370
120,2022-05-27,ads,6376,22449,112325,0.199858


- Sticky Factor, retentionRatio,%рекламного трафика, новые пользователи
- структура пользоваталей: новые активные спящие вернувщиеся и "дохлые"
- Регулярность активности(сообщений/лайков/просмотров) в день = общее кол-во/dau
- Frequency by Day - частота входов пользователей в зависимости от времени их первого визита
- K-factor = новые пользователи по каналу органика/(dau - новые пользователи)  - средне кол-во друзей, приглашенных одним активным пользователем


In [3]:
data = Getch(
    '''
    with t1 as (select user_id
                    ,source
                    ,toDate(time) as day
                    ,min(time) as first_date_time
                    ,max(time) as last_date_time
                    ,countIf(post_id,action ='like') as like
                    ,countIf(post_id,action='view') as view
                    ,countIf(post_id,action ='like')/countIf(post_id,action='view') as CTR
                from simulator_20220520.feed_actions
                group by user_id, day,source
    ),
    --отправка сообщений
    t2 as (select user_id
                ,source
                ,toDate(time) as day
                ,count(reciever_id) as send_message
                ,min(time) as first_date_time
                ,max(time) as last_date_time
            from simulator_20220520.message_actions
            group by user_id,day,source
    ),
    --получение сообщений
    t3 as (select reciever_id,toDate(time) as day
                ,count(user_id) as recieve_message
            from simulator_20220520.message_actions
            group by reciever_id,day
    ),
    --весь список пользователей
    t as (select user_id,gender,age,country,city,
            os, source, exp_group
            from simulator_20220520.feed_actions
            union distinct
            select user_id,gender,age,country,city,
            os, source, exp_group 
            from simulator_20220520.message_actions)
            
    select   if(t1.user_id = 0,t2.user_id,t1.user_id) as user_id
            , if(user_id = 0,t3.reciever_id,user_id) as reciever_id
            , if(t1.user_id = 0, 0, 1) as user_id_news
            , if(t2.user_id  = 0, 0, 1) as user_id_message
            , if(t1.user_id > 0 and t2.user_id > 0, 1,0) as user_id_news_message
            , t3.reciever_id as user_reciever_id
            , if(t1.source ='0',t2.source,t1.source) as source
            , if(t1.day ='1970.01.01',t2.first_date_time, if(t1.first_date_time
            
            ) as first_date_time
            , if(t1.day ='1970.01.01',t2.day,t1.day) as day
            , like
            , view
            , CTR
            --, percent_adv_source
            , send_message
            , recieve_message
    from t1
    full join t2 on t1.user_id = t2.user_id and t1.day = t2.day
    
    left join t3 on if(t1.user_id = 0,t2.user_id,t1.user_id)  = t3.reciever_id  
    and if(t1.day ='1970.01.01',t2.day,t1.day) = t3.day
    
    left join t on if(t1.user_id = 0,t2.user_id,t1.user_id) = t.user_id
    
    where day > today()-8
    '''
    ).df
data.head(2)

 b"Code: 62. DB::Exception: Syntax error: failed at position 1949 ('(') (line 44, col 17): (t1.day ='1970.01.01',t2.first_date_time, if(t1.first_date_time\n            \n            ) as first_date_time\n            , if(t1.day ='1970.01.01',t2.day,t1.da. Unmatched parentheses: (. (SYNTAX_ERROR) (version 21.12.2.17 (official build))\n"


NameError: name 'exit' is not defined

In [158]:
data_new = data.groupby('day', as_index = False).agg({'user_id':'nunique',
                                                      'user_id_news':'sum',
                                                      'user_id_message':'sum',
                                                      'user_id_news_message':'sum',
                                                      'reciever_id':'nunique',
                                                       'like':'sum',
                                                       'view':'sum',
                                                       'send_message':'sum',
                                                       'recieve_message':'sum'})
data_new['diff'] = data_new['send_message'] - data_new['recieve_message']
#data_new['uniq']
data_new

,day,user_id,user_id_news,user_id_message,user_id_news_message,reciever_id,like,view,send_message,recieve_message,diff
0,2022-05-20,20517,19129.0,1569.0,181.0,20517,112224,537657,12900,12837,63
1,2022-05-21,20717,18613.0,2373.0,269.0,20717,116868,564111,12888,12528,360
2,2022-05-22,20079,18572.0,1681.0,174.0,20079,102580,498156,12900,12842,58
3,2022-05-23,20462,19072.0,1612.0,222.0,20462,119012,574585,12899,12812,87
4,2022-05-24,21364,19211.0,2476.0,323.0,21364,135958,675874,12901,12812,89
5,2022-05-25,21821,20316.0,1739.0,234.0,21821,113263,583467,12899,12846,53
6,2022-05-26,21642,20187.0,1662.0,207.0,21642,100663,522682,12901,12806,95
7,2022-05-27,17038,15991.0,1169.0,122.0,17038,57251,288388,7195,7170,25


In [87]:
full_data = Getch(
    '''
    with t1 as (select user_id
                    ,toDate(time) as day
                    ,source
                    ,action
                    ,count(post_id) as action_value
                from simulator_20220520.feed_actions
                group by user_id, day,source, action
    ),
    t2 as (select user_id
                ,toDate(time) as day
                ,source
                ,'message' as action
                ,count(user_id) as action_value
            from simulator_20220520.message_actions
            group by user_id,day,source
    )
    select tt.* 
    from   (select user_id
                    ,source
                    ,day
                    ,sumIf(action_value,action ='like') as like
                    ,sumIf(action_value,action='view') as view
                    ,sumIf(action_value,action='message') as message
                    from (select * from t1 union all select * from t2) as  t
                    --where day > today()-8
                    group by user_id, source,day) tt
    --left join (select * from t1 union all select * from t2) as mau
    
    '''
    ).df
full_data.head(2)

,user_id,source,day,like,view,message
0,64243,ads,2022-05-24,7,40,0
1,124708,organic,2022-04-15,2,35,0


In [37]:
full_data_users = full_data.groupby('user_id', as_index = False)[['day']]\
                            .agg(['min','max'])\
#                            .rename(columns = {'min' : 'first_date','max':'last_date'}).reset_index()

stats = ['min','max']
full_data_users.columns = ['date_{}'.format(stat) for stat in stats]
full_data_users = full_data_users.reset_index()
full_data_users.head(2)

,user_id,date_min,date_max
0,200,2022-03-28,2022-05-25
1,201,2022-03-29,2022-05-21


In [24]:
full_data.shape
mapping = {True:1, False:0}

In [40]:
full_data = full_data.merge(full_data_users, how = 'left', on = 'user_id')
full_data.head(4)

,user_id,source,day,like,view,message,date_min,date_max
0,64243,ads,2022-05-24,7,40,0,2022-05-23,2022-05-25
1,124708,organic,2022-04-15,2,35,0,2022-04-15,2022-05-24
2,141419,organic,2022-05-11,7,24,0,2022-05-02,2022-05-23
3,30734,ads,2022-05-12,8,32,0,2022-04-28,2022-05-12


In [67]:
full_data['new_user'] = (full_data['day'] == full_data['date_min']).map(mapping)
full_data['user_ads'] = (full_data['source'] == 'ads').map(mapping)
full_data[full_data['new_user'] == 1]

,user_id,source,day,like,view,message,date_min,date_max,new_user,new_user_organic,user_organic,user_ads
1,124708,organic,2022-04-15,2,35,0,2022-04-15,2022-05-24,1,1,1,0
6,60297,ads,2022-05-19,4,21,0,2022-05-19,2022-05-20,1,0,0,1
11,54000,ads,2022-05-13,7,36,0,2022-05-13,2022-05-17,1,0,0,1
15,35064,ads,2022-05-02,15,110,0,2022-05-02,2022-05-22,1,0,0,1
23,138056,organic,2022-04-30,2,29,0,2022-04-30,2022-05-18,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
856644,130109,organic,2022-04-20,13,69,0,2022-04-20,2022-05-26,1,1,1,0
856651,53993,ads,2022-05-13,10,40,0,2022-05-13,2022-05-18,1,0,0,1
856654,130150,organic,2022-04-24,10,58,0,2022-04-24,2022-05-13,1,1,1,0
856657,60304,ads,2022-05-19,2,17,0,2022-05-19,2022-05-25,1,0,0,1


In [68]:
full_data['new_user_organic'] = full_data[['new_user','source']].apply(lambda x: 1 if (x['new_user'] == 1 and x['source'] == 'organic') else 0,axis=1)

In [69]:
full_data[full_data['new_user_organic'] == 1]

,user_id,source,day,like,view,message,date_min,date_max,new_user,new_user_organic,user_organic,user_ads
1,124708,organic,2022-04-15,2,35,0,2022-04-15,2022-05-24,1,1,1,0
23,138056,organic,2022-04-30,2,29,0,2022-04-30,2022-05-18,1,1,1,0
28,130084,organic,2022-04-20,6,19,0,2022-04-20,2022-05-27,1,1,1,0
38,116061,organic,2022-04-06,13,36,0,2022-04-06,2022-05-24,1,1,1,0
59,137991,organic,2022-04-28,7,33,0,2022-04-28,2022-05-23,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
856633,158188,organic,2022-05-18,3,17,0,2022-05-18,2022-05-24,1,1,1,0
856634,126378,organic,2022-04-23,2,24,0,2022-04-23,2022-04-28,1,1,1,0
856644,130109,organic,2022-04-20,13,69,0,2022-04-20,2022-05-26,1,1,1,0
856654,130150,organic,2022-04-24,10,58,0,2022-04-24,2022-05-13,1,1,1,0


In [ ]:
full_data_new = full_data.groupby('day', as_index = False).agg({'user_id':'nunique',
                                                      #'user_id_news':'sum',
                                                      #'user_id_message':'sum',
                                                      #'user_id_news_message':'sum',
                                                      #'reciever_id':'nunique',
                                                       'new_user':'sum',
                                                       'new_user_organic':'sum',
                                                       'user_ads':'sum',                                                        
                                                       'like':'sum',
                                                       'view':'sum',
                                                       'message':'sum'}).sort_values('day', ascending =True)\
                                                        .rename(columns = {'user_id' : 'dau'})
full_data_new['CTR'] = (full_data_new['like']/full_data_new['view']).round(2)
full_data_new['actions per user'] = ((full_data_new['view']+full_data_new['like']+full_data_new['message'])/full_data_new['dau']).round(0)
full_data_new['k-factor'] = (full_data_new['new_user_organic']/(full_data_new['dau']-full_data_new['new_user'])).round(4)
full_data_new['ads_traffic'] = (full_data_new['user_ads']/full_data_new['dau']).round(2)
full_data_new


,day,dau,new_user,new_user_organic,user_ads,like,view,message,CTR,actions per user,k-factor,ads_traffic
0,2022-03-28,1252,1252,553,699,1880,8208,3226,0.23,11.0,inf,0.56
1,2022-03-29,2681,1867,941,1375,13305,60906,12895,0.22,32.0,1.1560,0.51
2,2022-03-30,3699,2012,1014,1836,26996,125230,12897,0.22,45.0,0.6011,0.50
3,2022-03-31,4422,1815,960,2113,32498,150516,12897,0.22,44.0,0.3682,0.48
4,2022-04-01,5339,1889,1055,2467,37450,179308,12898,0.21,43.0,0.3058,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...
56,2022-05-23,20462,1971,928,8359,119012,574585,12899,0.21,35.0,0.0502,0.41
57,2022-05-24,21364,2083,1051,8754,135958,675874,12901,0.20,39.0,0.0545,0.41
58,2022-05-25,21821,2464,1189,9059,113263,583467,12899,0.19,33.0,0.0614,0.42
59,2022-05-26,21642,2176,982,8972,100663,522682,12901,0.19,29.0,0.0504,0.41


In [96]:
StickyFactor = Getch(
    '''
    with t1 as (select user_id
                    ,toDate(time) as day
                    ,source
                    ,action
                    ,count(post_id) as action_value
                from simulator_20220520.feed_actions
                group by user_id, day,source, action
    ),
    t2 as (select user_id
                ,toDate(time) as day
                ,source
                ,'message' as action
                ,count(user_id) as action_value
            from simulator_20220520.message_actions
            group by user_id,day,source
    )
    select tt.day
            ,tt.dau 
            ,count(distinct user_id)as mau
            ,dau/mau as sticky_factor
            ,count(distinct mau.day) as number_days
    from   (select  day
                    ,count(distinct user_id) as dau
                    from (select * from t1 union all select * from t2) as  t
                    group by day) tt
    cross join (select * from t1 union all select * from t2) as mau
    where mau.day >= tt.day-30 and mau.day<tt.day
    group by tt.day, tt.dau
    order by tt.day
    '''
    ).df
#full_data.head(2)

In [1]:
StickyFactor.head(20).sort_values('day', ascending = False)

NameError: name 'StickyFactor' is not defined